In [3]:
!pip install pyaudio



In [4]:
! pip install pydub simpleaudio




In [5]:
! pip install "assemblyai[extras]"

In [6]:
! pip install elevenlabs==0.3.0b0

In [7]:
! pip install mpv

In [8]:
! pip install --upgrade openai

In [9]:
import assemblyai as aai
from elevenlabs import generate,stream
from openai import OpenAI

In [ ]:
 import assemblyai as aai
from elevenlabs import generate,stream
from openai import OpenAI

class AI_Assistant:
    def __init__(self):
        aai.settings.api_key = "2810a6e1c2cd47c893288f82677f7d49"
        self.openai_client = OpenAI(api_key="2810a6e1c2cd47c893288f82677f7d49")
        self.elevenlabs_api_key = "2810a6e1c2cd47c893288f82677f7d49"
        self.transcriber = None
        self.full_transcript = [
            {
                "role": "system", "content": "You are a receptionist at a dental clinic. Be resourceful and efficient."
            },
        ]

    def start_transcription(self):
        self.transcriber = aai.RealtimeTranscriber(
            sample_rate=16000,
            on_data=self.on_data,
            on_error=self.on_error,
            on_open=self.on_open,
            on_close=self.on_close,
            end_utterance_silence_threshold=1000
        )
        self.transcriber.connect()
        microphone_stream = aai.extras.MicrophoneStream(sample_rate=16000)
        self.transcriber.stream(microphone_stream)
        
    def stop_transcription(self):
        if self.transcriber:
            self.transcriber.close()
            self.transcriber = None
            
    def on_open(self, session_opened: aai.RealtimeSessionOpened):
        return
        # print("Session ID:", session_opened.session_id)

    def on_data(self, transcript: aai.RealtimeTranscript):
        if not transcript.text:
            return

        if isinstance(transcript, aai.RealtimeFinalTranscript):
            self.generate_ai_response(transcript)
        else:
            print(transcript.text, end="\r")

    def on_error(self, error: aai.RealtimeError):
        return
        # print("An error occurred:", error)

    def on_close(self):
        return
        # print("Closing Session")

    def generate_ai_response(self, transcript):
        self.stop_transcription()
        self.full_transcript.append({
            "role": "user", "content": transcript.text
        })
        print(f"\nPatient: {transcript.text}", end="\r\n")
        response = self.openai_client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=self.full_transcript
        )
        ai_response = response.choices[0].message['content']
        self.generate_audio(ai_response)
        self.start_transcription()

    def generate_audio(self, text):
        self.full_transcript.append({"role": "assistant", "content": text})
        print(f"\nAI Receptionist: {text}")

        valid_voice_name = "Lily"  # Replace with a valid voice name
        try:
            audio_stream = generate(
                api_key=self.elevenlabs_api_key,
                text=text,
                voice=valid_voice_name,
                stream=True
            )
            stream(audio_stream)
        except ValueError as e:
            print(f"Error generating audio: {e}")

greeting = "Thank you for calling Alok Dental Clinic. My name is Garima, how may I assist you?"
ai_assistant = AI_Assistant()
ai_assistant.generate_audio(greeting)
ai_assistant.start_transcription()


